In [24]:
pip install dotenv pypdf langchain langchain-community langchain-openai langchain-pinecone docx2txt tiktoken  -q


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(), override=True)

True

In [3]:
pip install wikipedia -q 


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
def load_document(file):
    import os

    name, extension = os.path.splitext(file)

    if extension == ".pdf":
        from langchain_classic.document_loaders import PyPDFLoader

        print(f"Loading {file}")
        loader = PyPDFLoader(file)
    elif extension == ".docx":
        from langchain_classic.document_loaders import Docx2txtLoader

        loader = Docx2txtLoader(file)
    else:
        print("Document format is not supported")
        return None

    data = loader.load()
    return data


# wikipedia


def load_from_wikipedia(query, lang="en", load_max_docs=2):
    from langchain_classic.document_loaders import WikipediaLoader

    loader = WikipediaLoader(query=query, lang=lang, load_max_docs=load_max_docs)
    data = loader.load()
    return data

In [5]:
def chunk_data(data, chunk_size=256):
    from langchain_classic.text_splitter import RecursiveCharacterTextSplitter

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=0
    )
    chunks = text_splitter.split_documents(data)
    return chunks

In [11]:
def print_embedding_cost(texts):
    import tiktoken

    enc = tiktoken.encoding_for_model("text-embedding-ada-002")
    total_tokens = sum([len(enc.encode(page.page_content)) for page in texts])
    print(f"Total Tokens: {total_tokens}")
    print(f"Embedding Cost in USD: {total_tokens / 1000 * 0.00002:.6f}")

### Embedding and uploading to a Vector Database (Pinecone)

In [26]:
def insert_or_fetch_embeddings(index_name, chunks):
    import pinecone

    # from langchain_community.vectorstores import Pinecone
    from langchain_pinecone.vectorstores import Pinecone
    from langchain_openai import OpenAIEmbeddings
    from pinecone import ServerlessSpec

    pc = pinecone.Pinecone()
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small", dimensions=1536)

    if index_name in pc.list_indexes().names():
        print(f"Index {index_name} already exists. Loading embeddings...", end="")
        vector_store = Pinecone.from_existing_index(index_name, embeddings)
        print("Ok")
    else:
        print(f"Creating index {index_name} and embeddings...", end="")
        pc.create_index(
            name=index_name,
            dimension=1536,
            metric="cosine",
            spec=ServerlessSpec(cloud="aws", region="us-east-1"),
        )
        vector_store = Pinecone.from_documents(
            chunks, embeddings, index_name=index_name
        )
        print("Ok")
        return vector_store

In [16]:
def delete_pinecone_index(index_name="all"):
    import pinecone

    pc = pinecone.Pinecone()
    if index_name == "all":
        indexes = pc.list_indexes().names()
        print("Deleting all indexes...")
        for index in indexes:
            pc.delete_index(index)
        print("Ok")
    else:
        print(f"Deleting index {index_name}...", end="")
        pc.delete_index(index_name)
        print("Ok")

## Running code

In [18]:
data = load_document("pdfs/guia_lgpd.pdf")
# print(data[1].page_content)

print(f"You have {len(data)} pages in your data")

Loading pdfs/guia_lgpd.pdf
You have 69 pages in your data


In [8]:
# data = load_from_wikipedia("Gemini-AI")
# print(data[0].page_content)

In [19]:
chunks = chunk_data(data)
print(len(chunks))
print(chunks[10].page_content)

929
HISTÓRICO DE VERSÕES
DATA VERSÃO DESCRIÇÃO AUTOR
23/03/2020 1.0 Primeira versão do Guia de Boas Práticas. Equipe Técnica de 
Elaboração
14/08/2020 2.0
Segunda versão do Guia de Boas Práticas 
considerando as sugestões enviadas pelo:


In [20]:
print_embedding_cost(chunks)

Total Tokens: 52383
Embedding Cost in USD: 0.001048


In [27]:
delete_pinecone_index()

Deleting all indexes...
Ok


In [28]:
index_name = "askadocument"
vector_store = insert_or_fetch_embeddings(index_name, chunks)

Creating index askadocument and embeddings...Ok
